## Sending Custom Logs and Metrics To Datadog

### Environment Setup
Install packages.
>Secrets (API keys) **ARE USED** in this demo.

To use this code you'll need to upload a `.env` file to the root directory for this notebook. Click on `File Browser` (folder) icon to view the directory. All files starting with a period are hidden by default. Click the eye icon to view them.

View the [README](https://console.cloud.google.com/vertex-ai/colab/notebooks?project=vertexaiwebinar&supportedpurview=project&activeNb=projects%2Fvertexaiwebinar%2Flocations%2Fus-central1%2Frepositories%2Fccc4213c-e5f1-4017-a1ef-f1984b49b4dc) for information on how to obtain and use your own API keys from an `.env` file.



In [ ]:
!pip install python-dotenv
!pip install yfinance
!pip install faker

### Send Custom Logs to the Datadog API Endpoints

In [ ]:
import requests
import json

# Used to pull in API Keys/Secrets
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()
DD_API_KEY = os.getenv("DD_API_KEY")

# Datadog Log API endpoint
url = 'https://http-intake.logs.datadoghq.com/v1/input'

# Custom log entry
log_entry = {
    'message': 'Hello from Vertex AI',
    'ddtags': 'env:development,version:1.0.1',
    'hostname': 'gcp-vertex-ai',
    'service': 'webinar',
    'ddsource': 'python-colab-notebook'
}

headers = {
    'Content-Type': 'application/json',
    'DD-API-KEY': DD_API_KEY
}

# Convert log entry to JSON and print it
json_log_entry = json.dumps(log_entry)
print("Log Entry to be sent to Datadog:")
print(json_log_entry)

response = requests.post(url, headers=headers, data=json.dumps(log_entry))

# Check response from Datadog
if response.status_code == 200:
    print('Log sent successfully')
else:
    print(f'Failed to send log: {response.text}')

### Send "Hello from NASA (and Vertex AI) Custom Log to Datadog
This code calls the NASA image endpoint and then sends that data to the Datadog Logs API which can then be viewed in Datadog.

>**NOTE:** Calling the custom NASA image (GCP Cloud Function) endpoint allows me to trigger my own app which can then be viewed in Datadog.

In [ ]:
import requests
import json

# Used to pull in API Keys/Secrets
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()
DD_API_KEY = os.getenv("DD_API_KEY")

# Datadog Log API endpoint
dd_url = 'https://http-intake.logs.datadoghq.com/v1/input'

# NASA Image API endpoint
nasa_url = 'https://us-central1-datadog-community.cloudfunctions.net/get-nasa-image'

# Get NASA image data (assuming it returns text)
nasa_response = requests.get(nasa_url)

# Check response from NASA API and store the text
nasa_text = ''
if nasa_response.status_code == 200:
    nasa_text = nasa_response.text
    print("\nNASA Image Data (Text):")
    print(nasa_text)
else:
    print(f'Failed to retrieve NASA image data: {nasa_response.text}')

# Custom log entry including NASA data
log_entry = {
    'message': 'Hello from NASA (and Vertex AI). Here is the latest NASA image data:',
    'nasa_data': nasa_text,  # Include NASA data as a custom field
    'ddtags': 'env:development,version:1.1.0',
    'hostname': 'gcp-vertex-ai',
    'service': 'webinar',
    'ddsource': 'python-colab-notebook'
}

headers = {
    'Content-Type': 'application/json',
    'DD-API-KEY': DD_API_KEY
}

# Convert log entry to JSON and print it
json_log_entry = json.dumps(log_entry)
print("#####################")
print("Log Entry to be sent to Datadog:")
print(json_log_entry)

# Send the log to Datadog
dd_response = requests.post(dd_url, headers=headers, data=json_log_entry)

# Check response from Datadog
if dd_response.status_code == 200:
    print("#####################")
    print('Log sent successfully')
else:
    print(f'Failed to send log: {dd_response.text}')


### Send Financial Market Custom Metrics to Datadog
Using the `yfinance` open source library, look up historical data for multiple tickers and then save that information in json format so that it can be sent to datadog's metrics endpoint

In [ ]:
import requests
import yfinance as yf
import json
import datetime

# Used to pull in API Keys/Secrets
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()
DD_API_KEY = os.getenv("DD_API_KEY")

# Datadog Metrics API endpoint
dd_url = 'https://api.datadoghq.com/api/v1/series'

# List of tickers you want to look up
tickers = ["DJIA", "NDX"]

# Define the time period for historical data
period = "1mo"

# Fetch historical data for all tickers
historical_data = {}
for ticker in tickers:
    stock = yf.Ticker(ticker)
    data = stock.history(period=period)
    # Convert the data to a format suitable for sending as a metric
    # Here we're just grabbing the Close prices for simplicity
    historical_data[ticker] = data['Close'].tolist()

# Prepare data in the format expected by Datadog's metrics endpoint
datadog_metrics = {
    "series": []
}

current_time = int(datetime.datetime.now().timestamp())

for ticker, prices in historical_data.items():
    for i, price in enumerate(prices):
        datadog_metrics["series"].append({
            "metric": f"stock.{ticker.lower()}.close",
            "points": [[current_time, price]],
            "type": "gauge",
            "host": "gcp-vertex-ai",
            "tags": [f"ticker:{ticker}"]
        })

# Convert to JSON
json_data = json.dumps(datadog_metrics)

# Headers for the Datadog API request
headers = {
    'Content-Type': 'application/json',
    'DD-API-KEY': DD_API_KEY
}

# Send the data to Datadog's metrics endpoint
response = requests.post(dd_url, headers=headers, data=json_data)

# Check response from Datadog
if response.status_code == 202:
    print('Data sent successfully to Datadog')
    print(json_data)
else:
    print(f'Failed to send data to Datadog: {response.text}')


### Send Random Data to Datadog Metrics API

Send randomized metrics to Datadog.

In [ ]:
import requests
import json
import datetime
import random

# Used to pull in API Keys/Secrets
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
DD_API_KEY = os.getenv("DD_API_KEY")

# Datadog Metrics API endpoint
dd_url = 'https://api.datadoghq.com/api/v1/series'

# Prepare data in the format expected by Datadog's metrics endpoint
datadog_metrics = {
    "series": []
}

current_time = int(datetime.datetime.now().timestamp())

# Define your custom metrics and their historical data
# Randomly generate historical data for this example
historical_data = {
    "vertex_metric1": [random.uniform(10, 90) for _ in range(5)],
    "vertex_metric2": [random.uniform(20, 60) for _ in range(5)],
    "vertex_metric3": [random.uniform(30, 70) for _ in range(5)],
    "vertex_metric4": [random.uniform(55, 90) for _ in range(5)],
    "vertex_metric5": [random.uniform(35, 75) for _ in range(5)]
}

for custom, values in historical_data.items():
    for i, value in enumerate(values):
        # Generate random coordinates for each data point
        random_latitude = round(random.uniform(-90, 90), 6)
        random_longitude = round(random.uniform(-180, 180), 6)

        # Generate a random value for each metric
        random_value = round(value, 2)  # Round to 2 decimal places for readability
        datadog_metrics["series"].append({
            "metric": f"custom.{custom.lower()}.value",
            "points": [[current_time - i*60, random_value]],  # Simulate data for each minute
            "type": "interval",
            "host": "vertex.ai",
            "tags": [f"custom:{custom}", f"latitude:{random_latitude}", f"longitude:{random_longitude}"]
        })

# Convert to JSON
json_data = json.dumps(datadog_metrics)

# Headers for the Datadog API request
headers = {
    'Content-Type': 'application/json',
    'DD-API-KEY': DD_API_KEY
}

# Send the data to Datadog's metrics endpoint
response = requests.post(dd_url, headers=headers, data=json_data)

# Print the response code from Datadog
print(f"Response Code from Datadog: {response.status_code}")

# Check response from Datadog
if response.status_code == 202:
    print('Data sent successfully to Datadog')
    print(json_data)
else:
    print(f'Failed to send data to Datadog: {response.text}')


### Send Random Coordinates to Datadog Logs API

In [ ]:
import requests
import json
import random
from dotenv import load_dotenv
import os

# Function to generate a random log entry with latitude, longitude, and additional metrics
def generate_log_entry():
    random_number = random.randint(1, 5)  # Generate a random number between 1 and 10
    log_entry = {
        'message': 'Random Global Coordinates',
        'ddtags': 'env:development,version:1.0.1',
        'hostname': f'global-host-{random_number}',  # Append random number to hostname
        'service': f'global-coordinates-{random_number}',  # Append random number to service
        'ddsource': 'vertex-ai',
        'latitude': round(random.uniform(-90, 90), 6),  # Random latitude
        'longitude': round(random.uniform(-180, 180), 6),  # Random longitude
        'value': random.randint(0, 500000),  # Random number between 0 and 500,000
    }

    # Add 5 additional metrics with random values between 0 and 100
    for i in range(1, 6):
        log_entry[f"metric{i}"] = round(random.uniform(0, 100), 2)

    return log_entry

# Load environment variables from .env file
load_dotenv()
DD_API_KEY = os.getenv("DD_API_KEY")

# Datadog Log API endpoint
url = 'https://http-intake.logs.datadoghq.com/v1/input'

# Generate a random log entry
log_entry = generate_log_entry()

headers = {
    'Content-Type': 'application/json',
    'DD-API-KEY': DD_API_KEY
}

# Convert log entry to JSON
json_log_entry = json.dumps(log_entry)
print("Log Entry to be sent to Datadog:")
print(json_log_entry)

# Send the log entry to Datadog
response = requests.post(url, headers=headers, data=json_log_entry)

# Check response from Datadog
if response.status_code == 200:
    print('Log sent successfully')
else:
    print(f'Failed to send log: {response.text}')


### Experimenting with the Text-Bison-32K model

Using the text bison model generate a box score for the Indianapolos 500 with fictional names. Include the car number, driver's full name, driver's home country, the number of laps completed (assuming not all cars will finish and receieve a DNF). Include the lap speed of each driver for each of the 200 laps.

#### Fictional Race Using **Text-Bison-32k** Model

AI Generated boxscore of a fictional Indianapolis 500 race. 🏎️ 🏁

In [ ]:
import requests
import vertexai
from vertexai.preview.language_models import TextGenerationModel
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
project_id = os.getenv("project_id")

# Constants
MODEL_NAME = 'text-bison-32k'  # Replace with the actual model name you have access to

# Function Definitions
def generate_indianapolis_500_box_score(project_id, model_name):
    """
    Generate a fictional box score for the Indianapolis 500 with fictional driver names. Do not use real names.
    Parameters:
        project_id (str): Project ID for Vertex AI.
        model_name (str): The name of the model to use for generation.
    """
    prompt = "Generate a fictional box score for the Indianapolis 500 with fictional driver names. Do not use real names."

    vertexai.init(project=project_id)
    model = TextGenerationModel.from_pretrained(model_name)
    response = model.predict(prompt, temperature=0.2, max_output_tokens=3000)

    # Print the generated box score
    print("Generated Indianapolis 500 Box Score:")
    print(response.text)

# Main Script
def main():
    generate_indianapolis_500_box_score(project_id, MODEL_NAME)

if __name__ == "__main__":
    main()


#### Fictional Race Using **Gemini Pro** Model

Like above, but stores the results in a variable - to be (possibly) used in a subsequent cell.

In [ ]:
# Import necessary libraries
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

# Initialize Vertex AI with your project ID
vertexai.init(project='vertexaiwebinar')  # Replace with your project ID

# Prompt to send to Vertex AI
prompt = "Generate a fictional box score for the Indianapolis 500 with fictional driver names. Do not use real names."

# Language to use
language = "en-US"
# Model name to use
model_name = "gemini-pro"  # Replace with the actual model name you have access to

def generate_and_display(model_name, full_prompt):
    model = GenerativeModel(model_name)
    responses = model.generate_content(
        full_prompt,  # Using the combined prompt
        generation_config={
            "max_output_tokens": 2048,
            "temperature": 0.99,
            "top_p": 1
        },
        stream=True,
    )
    generated_texts = []
    for response in responses:
        generated_text = response.candidates[0].content.parts[0].text
        print(generated_text)  # Display the generated text as output
        generated_texts.append(generated_text)
    return generated_texts

# Call the generate_and_display function with the user-defined model name and the full prompt
# and save the results to a variable
generated_box_scores = generate_and_display(model_name, prompt)

# Now, `generated_box_scores` contains the generated text and can be used in future code snippets.
# The generated text is also printed as output.


#### View the Live Tail in Datadog

View logs as they arrive in the Datadog [Live Tail View](https://app.datadoghq.com/logs/livetail?query=&view=spans).

Send lap times to Datadog for random named drivers

#### Sending **Logs** Data for Each Lap


In [ ]:
import requests
import json
import random
import time
from faker import Faker  # Library to generate fake data
from dotenv import load_dotenv
import os

# Initialize Faker
fake = Faker()

# Load environment variables from .env file
load_dotenv()
DD_API_KEY = os.getenv("DD_API_KEY")

# Datadog Log API endpoint
url = 'https://http-intake.logs.datadoghq.com/v1/input'

# Number of cars and laps
num_cars = 10
num_laps = 3

def generate_random_lap_data(car_number, lap_number):
    """Generate random data for a car and lap based on the schema."""
    return {
        "ddsource": "Indy 500",
        "ddtags": "env:production,version:1.0",
        "hostname": f"car-{car_number}-indianapolis-500-results",
        "message": f"Lap {lap_number} completed",
        "service": f"{car_number}-race_results",
        "driver": fake.name(),
        "carNumber": car_number,
        "lapNumber": lap_number,
        "averageLapSpeed": round(random.uniform(150, 230), 2)  # Generate random speed
    }

def send_to_datadog(payload):
    """Send the provided JSON payload as a custom log to Datadog."""
    headers = {
        'Content-Type': 'application/json',
        'DD-API-KEY': DD_API_KEY
    }
    json_payload = json.dumps(payload)
    response = requests.post(url, headers=headers, data=json_payload)
    if response.status_code == 200:
        print(f'Log for Car {payload["carNumber"]} on Lap {payload["lapNumber"]} sent successfully to Datadog')
    else:
        print(f'Failed to send log to Datadog: {response.text}')

# Loop through each lap, generating and sending log for each car
for lap in range(1, num_laps + 1):
    for car in range(1, num_cars + 1):
        lap_log = generate_random_lap_data(car, lap)
        send_to_datadog(lap_log)
        time.sleep(1)  # Wait for 1 second before sending the next log

print("All lap logs have been sent to Datadog.")


#### Sending Metrics Data for Each Lap

In [ ]:
import requests
import json
import random
import time
from faker import Faker  # Library to generate fake data
from dotenv import load_dotenv
import os

# Initialize Faker
fake = Faker()

# Load environment variables from .env file
load_dotenv()
DD_API_KEY = os.getenv("DD_API_KEY")

# Datadog Metrics API endpoint
url = 'https://api.datadoghq.com/api/v1/series'

# Number of cars and laps
num_cars = 10
num_laps = 3

def generate_metric_data(car_number, lap_number):
    """Generate metric data for a car and lap."""
    return {
        "metric": "race.lap.speed",
        "points": [[int(time.time()), round(random.uniform(150, 230), 2)]],
        "type": "gauge",
        "host": f"car-{car_number}-indianapolis-500-results",
        "tags": [
            f"env:production",
            f"version:1.0",
            f"driver:{fake.name()}",
            f"car_number:{car_number}",
            f"lap_number:{lap_number}"
        ]
    }

def send_to_datadog(metrics):
    """Send the provided metric data to Datadog."""
    headers = {
        'Content-Type': 'application/json',
        'DD-API-KEY': DD_API_KEY
    }
    payload = {
        "series": metrics
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    if response.status_code == 202:
        print('Metric data sent successfully to Datadog')
    else:
        print(f'Failed to send metric data to Datadog: {response.text}')

# Loop through each lap, generating and sending metric data for each car
for lap in range(1, num_laps + 1):
    metrics = []
    for car in range(1, num_cars + 1):
        metric_data = generate_metric_data(car, lap)
        # Display the metric data on the screen
        print(f"Generated Metric Data for Car {car} on Lap {lap}: {json.dumps(metric_data, indent=2)}")
        metrics.append(metric_data)
    send_to_datadog(metrics)
    time.sleep(1)  # Wait for 1 second before sending the next set of metrics

print("All metric data have been sent to Datadog.")


#### Sends Metrics & Logs At the Same Time
In order to see correlation.
>NOTE: Open the Live Tail before running this.

In [ ]:
import requests
import vertexai
from vertexai.preview.language_models import TextGenerationModel
import json
import random
import time
from dotenv import load_dotenv
import os
import re

# Load environment variables from .env file
load_dotenv()
DD_API_KEY = os.getenv("DD_API_KEY")
VERTEX_PROJECT_ID = os.getenv("VERTEX_PROJECT_ID")  # Your Vertex AI project ID

# Datadog Metrics and Logs API endpoints
datadog_metrics_url = 'https://api.datadoghq.com/api/v1/series'
datadog_logs_url = 'https://http-intake.logs.datadoghq.com/v1/input'

# Model name to use
MODEL_NAME = 'text-bison-32k'  # Replace with the actual model name you have access to

def generate_driver_names(project_id, model_name):
    """Generate a list of fictional race car driver names."""
    prompt = "Generate a list of 5 fictional race car driver names. The names should be a combination of a modern real driver and dragon fantasy."
    vertexai.init(project=project_id)
    model = TextGenerationModel.from_pretrained(model_name)
    response = model.predict(prompt, temperature=0.8, max_output_tokens=3000)

    names_text = response.text
    # Use regex to remove numbers and unwanted characters, then strip whitespace
    driver_names = [re.sub(r'[^A-Za-z ]', '', name).strip() for name in names_text.split('\n') if name.strip()]
    # Print the generated and cleaned driver names
    print("Cleaned Driver Names:")
    print(json.dumps(driver_names, indent=2))
    return driver_names  # Return the list of cleaned names

def generate_metric_data(driver_name, car_number, lap_number):
    """Generate metric data for a car and lap."""
    return {
        "metric": "race.lap.speed",
        "points": [[int(time.time()), round(random.uniform(150, 230), 2)]],
        "type": "gauge",
        "host": "dragon-rider-results",
        "tags": [
            f"env:production",
            f"version:1.0",
            f"driver:{driver_name}",
            f"car_number:{car_number}",
            f"lap_number:{lap_number}"
        ]
    }

def send_to_datadog_metrics(metrics):
    """Send the provided metric data to Datadog."""
    headers = {
        'Content-Type': 'application/json',
        'DD-API-KEY': DD_API_KEY
    }
    payload = {
        "series": metrics
    }
    # Print the metrics being sent to Datadog
    print("Sending the following metric data to Datadog:")
    print(json.dumps(payload, indent=2))
    response = requests.post(datadog_metrics_url, headers=headers, data=json.dumps(payload))
    if response.status_code == 202:
        print('Metric data sent successfully to Datadog')
    else:
        print(f'Failed to send metric data to Datadog: {response.text}')

def send_to_datadog_logs(log_message):
    """Send a custom log message to Datadog."""
    headers = {
        'Content-Type': 'application/json',
        'DD-API-KEY': DD_API_KEY
    }
    response = requests.post(datadog_logs_url, headers=headers, data=json.dumps(log_message))
    if response.status_code == 200:
        print('Log message sent successfully to Datadog')
    else:
        print(f'Failed to send log message to Datadog: {response.text}')

# Generate fictional driver names
driver_names = generate_driver_names(VERTEX_PROJECT_ID, MODEL_NAME)

# Number of laps (assuming 10 for this example)
num_laps = 10

# Loop through each lap, generating and sending metric data for each driver
for lap in range(1, num_laps + 1):
    metrics = []
    for i, driver_name in enumerate(driver_names):
        metric_data = generate_metric_data(driver_name, i + 1, lap)  # Car numbers are 1-indexed
        metrics.append(metric_data)
        # Construct a custom log message with additional information
        log_message = {
            "ddsource": "custom_metric_log",
            "ddtags": "env:production,version:1.0",
            "hostname": f"car-{i + 1}-lap-{lap}-dragon-racer-results",
            "message": f"Driver Metrics: {driver_name}, car number: {i + 1}, lap number: {lap}",
            "service": f"{driver_name}-race_metrics",
            "metric_info": metric_data
        }
    send_to_datadog_logs(log_message)
    send_to_datadog_metrics(metrics)
    # Wait for a random time up to 3 seconds before sending the next set of metrics
    time.sleep(random.uniform(0, 3))

print("All metric data and logs have been sent to Datadog.")
